# Variational autoencoder

In [1]:
using Flux, Flux.Data.MNIST
using Flux: @epochs, throttle, params
using Distributions
import Distributions: logpdf

**For numerically stable, extend distributions slightly to have a logpdf for `p` close to 1 or 0.**

In [2]:
logpdf(b::Bernoulli, y::Bool) = y * log(b.p + eps()) + (1 - y) * log(1 - b.p + eps())

logpdf (generic function with 62 methods)

# Load data

In [3]:
X = MNIST.images();

## Binarise

In [4]:
X = float.(hcat(vec.(X)...)) .> 0.5

784×60000 BitArray{2}:
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false  …  false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
     ⋮                      

## Mini-batches

In [5]:
N, M = size(X, 2), 100
data = [X[:,i] for i in Iterators.partition(1:N,M)]

600-element Array{BitArray{2},1}:
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false; … ; false false … false false; false false … false false]
 [false false … false false; false false … false false

# Define Model

In [6]:
# Dimensions (D -> Dh -> Dz -> Dh -> D)
Dz, Dh, D = 5, 500, 28^2

# Encoder
A, μ, logσ = Dense(D, Dh, tanh), Dense(Dh, Dz), Dense(Dh, Dz)
g(X) = (h = A(X); (μ(h), logσ(h)))
z(μ, logσ) = μ + exp(logσ) * randn()

# Decoder
f = Chain(Dense(Dz, Dh, tanh), Dense(Dh, D, σ))

Chain(Dense(5, 500, tanh), Dense(500, 784, NNlib.σ))

# Loss function and ELBO

In [7]:
# KL-divergence between approximation posterior and N(0, 1) prior.
kl_q_p(μ, logσ) = 0.5 * sum(exp.(2 .* logσ) + μ.^2 .- 1 .+ logσ.^2)

# logp(x|z) - conditional probability of data given latents.
logp_x_z(x, z) = sum(logpdf.(Bernoulli.(f(z)), x))

# Monte Carlo estimator of mean ELBO using M samples.
L̄(X) = ((μ̂, logσ̂) = g(X); (logp_x_z(X, z.(μ̂, logσ̂)) - kl_q_p(μ̂, logσ̂)) / M)

loss(X) = -L̄(X) + 0.01 * sum(x->sum(x.^2), params(f))

# Sample from the learned model.
modelsample() = rand.(Bernoulli.(f(z.(zeros(Dz), zeros(Dz)))))

modelsample (generic function with 1 method)

# Learning

In [8]:
evalcb = throttle(() -> @show(-L̄(X[:, rand(1:N, M)])), 30)
opt = ADAM(params(A, μ, logσ, f))
@epochs 10 Flux.train!(loss, zip(data), opt, cb=evalcb)

┌ Info: Epoch 1
└ @ Main /home/pika/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93


-(L̄(X[:, rand(1:N, M)])) = 542.0929128283901 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 205.08832708267968 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 206.43163041940676 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 181.38805021263047 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 185.15896372249222 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 174.9184993861609 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 215.1296003483748 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 176.91684689593768 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 179.122474319477 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 180.98286084084492 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 179.37794052383742 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 177.09325260981902 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 174.00850289109755 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 197.59559573942732 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 171.95256627032757 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 170.05080043329653 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 167.28943506765296 (tracked)


┌ Info: Epoch 2
└ @ Main /home/pika/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93


-(L̄(X[:, rand(1:N, M)])) = 168.1690423427329 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 163.68600247944403 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 167.88095225489644 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 162.73749244302337 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 171.8528419659619 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 161.33095713054314 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 161.76333643397456 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 169.1126078009159 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 168.7599280162136 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 160.1875805625217 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 163.73816553498938 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 160.98363750796742 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 158.7329207990596 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 160.48544402634593 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 164.38535738452575 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 156.3895800110892 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 156.01398422555522 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 150.195909

┌ Info: Epoch 3
└ @ Main /home/pika/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93


...

-(L̄(X[:, rand(1:N, M)])) = 139.11206863939242 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 144.91808872644856 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 140.4021858324071 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 144.28464967714044 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 141.4348317637279 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 142.687456995621 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 135.4119736237354 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 142.8763791243594 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 147.93450297858726 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 146.6739343173246 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 147.15512975051698 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 137.72001398373612 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 148.3405668535239 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 131.6126510622325 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 149.485785379957 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 141.6196108069408 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 139.66397761530564 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 141.61014476553

┌ Info: Epoch 9
└ @ Main /home/pika/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93


-(L̄(X[:, rand(1:N, M)])) = 132.97107074322946 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 139.82380012162855 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 148.19628431710896 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 141.78914167545847 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 145.83798394084272 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 136.63728904646268 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 144.04774098883559 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 129.4133392661895 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 153.4273533254705 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 146.4512702046793 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 144.10743648667446 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 139.66101003383645 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 138.2356980218178 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 141.40592359720006 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 140.43334738333658 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 136.87264699114462 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 145.2556824884992 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 139.7018

┌ Info: Epoch 10
└ @ Main /home/pika/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93


-(L̄(X[:, rand(1:N, M)])) = 134.10255477322974 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 138.12326341688254 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 148.69689151047191 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 140.27286575320394 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 131.82066912697522 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 131.61621000402707 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 139.21766624812915 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 143.0022087346481 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 141.56783694043352 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 134.93557489788893 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 141.05538176062032 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 149.3557305959455 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 132.8707477605601 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 141.74805251690756 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 137.75329347434334 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 152.1999455134351 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 135.60470290092246 (tracked)
-(L̄(X[:, rand(1:N, M)])) = 138.727

# Sample Output

In [9]:
using Images

img(x) = Gray.(reshape(x, 28, 28))

cd(@__DIR__)
sample = hcat(img.([modelsample() for i = 1:10])...)
save("vae-sample.png", sample)